In [29]:
pwd

'C:\\Users\\user\\Downloads\\IBM\\Drug classification using ML (2)\\Drug classification using ML\\training_ibm'

In [30]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
import warnings
import pickle
from scipy import stats
warnings.filterwarnings('ignore')
plt.style.use('fivethirtyeight')

In [31]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_db3aab955d234e4090c6889e4becd5ab = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='2-IeAj3ef7WK7cyYoO9Mk6UeNtpVfrbt-MGswj42wDpY',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

body = client_db3aab955d234e4090c6889e4becd5ab.get_object(Bucket='drugclassification-donotdelete-pr-jxnr6l87fbqazi',Key='drug200.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)
df_data_1.head()


In [32]:
# Shape of csv data
df_data_1.shape

NameError: name 'df_data_1' is not defined

In [ ]:
# Finding null values
df_data_1.isnull().sum()

In [ ]:
# Checking the information of features
df_data_1.info()

# Descriptive analysis

In [ ]:
df_data_1.describe(include='all')

# Univariate analysis

In [ ]:
# Checking the distribution (normal or skewed)

plt.figure(figsize=(12,5))
plt.subplot(121)
sns.distplot(df_data_1['Age'],color='r')
plt.subplot(122)
sns.distplot(df_data_1['Na_to_K'])
plt.show()

In [ ]:
# From the above plot age column is normally distributed. Na_to_k is right skewed (mean>mode). To overcome skewness transformation techniques can be used.

print(stats.mode(df_data_1['Na_to_K']))
print(np.mean(df_data_1['Na_to_K']))

In [ ]:
# Finding outliers

plt.figure(figsize=(12,5))
plt.subplot(121)
sns.boxplot(df_data_1['Age'])
plt.subplot(122)
sns.boxplot(df_data_1['Na_to_K'])
plt.show()

In [ ]:
# Na_to_K has 8 outliers. In this project we are not going to handle outliers. Most of the classification algorithms are not sensitive to outliers.

q1 = np.quantile(df_data_1['Na_to_K'],0.25)
q3 = np.quantile(df_data_1['Na_to_K'],0.75)

IQR = q3-q1

upper_bound = q3+(1.5*IQR)

print('Upper Bound :',upper_bound)

print('Skewed data :',len(df_data_1[df_data_1['Na_to_K']>upper_bound]))

In [ ]:
# Creating a data frame with categorical features for following visualization

df_cat = df_data_1.select_dtypes(include='object')
df_cat.head()

In [ ]:
# Visualizing the count of categorical variable.

plt.figure(figsize=(18,4))
for i,j in enumerate(df_cat):
    plt.subplot(1,4,i+1)
    sns.countplot(df_data_1[j])   

# Bivariate analysis

In [ ]:
df_data_1.head()

In [ ]:
# Visualizing the relation between drug, BP, sex & cholesterol

plt.figure(figsize=(20,5))
plt.subplot(131)
sns.countplot(df_data_1['Drug'],hue=df_data_1['BP'])
plt.legend(loc='upper right')
plt.subplot(132)
sns.countplot(df_data_1['Drug'],hue=df_data_1['Sex'])
plt.subplot(133)
sns.countplot(df_data_1['Drug'],hue=df_data_1['Cholesterol'])

In [ ]:
# Creating a new column Age_. This column shows the categorized age.

df_data_1['Age_'] = ['15-30' if x<=30 else '30-50' if x>30 and x<=50 else '50-75' for x in df_data_1['Age']]
df_data_1.head()

In [ ]:
# Finding the relation between categorized age and drug

pd.crosstab(df_data_1['Age_'],[df_data_1['Drug']])

In [ ]:
# Removing the Age_ column

df_data_1.drop('Age_',axis=1,inplace=True)
df_data_1.head()

# Multivariate Analysis

In [ ]:
sns.swarmplot(df_data_1['Drug'],df_data_1['Na_to_K'],hue=df_data_1['BP'])

# DrugC is used for low BP patient, DrugY is used on patients having Na_to_K > 15.

# Encoding- Converting categorical to numerical

In [ ]:
# Replacing low, normal & high with 0, 1 & 2...

df_data_1['BP'] = [0 if x=='LOW' else 1 if x=='NORMAL' else 2 for x in df_data_1['BP']]

In [ ]:
# Replacing normal and high cholesterol with 0 & 1

df_data_1['Cholesterol'] = [0 if x=='NORMAL' else 1 for x in df_data_1['Cholesterol']]

In [ ]:
# Replacing female and male with 0 & 1

df_data_1['Sex'] = [0 if x=='F' else 1 for x in df_data_1['Sex']]

In [ ]:
df_data_1.head()

In [ ]:
df_data_1['Drug'].value_counts()

In [ ]:
x = df_data_1.drop('Drug',axis=1)
x.head()

In [ ]:
y = df_data_1['Drug']
y.head()

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=10)

In [ ]:
print('Shape of x_train {}'.format(x_train.shape))
print('Shape of y_train {}'.format(y_train.shape))
print('Shape of x_test {}'.format(x_test.shape))
print('Shape of y_test {}'.format(y_test.shape))

In [ ]:
rf = RandomForestClassifier()
rf.fit(x_train,y_train)

In [ ]:
ypred = rf.predict(x_test)

In [ ]:
confusion_matrix(y_test,ypred)

In [ ]:
print(classification_report(y_test,ypred))

In [ ]:
pickle.dump(rf,open('model.pkl','wb'))

In [ ]:
!tar -zcvf drug-classification-model_new.tgz model.pkl

In [ ]:
ls -1

In [ ]:
!pip install watson-machine-learning-client --upgrade

In [ ]:
from ibm_watson_machine_learning import APIClient
wml_credentials = {
    "url": "https://us-south.ml.cloud.ibm.com",
    "apikey": "NW6_YjwnxreHnFE-dgSFLYScfWDFhMW2WOY0qggTfdns"
}
client = APIClient(wml_credentials)

In [ ]:
def guid_from_space_name(client, space_name):
    space = client.spaces.get_details()
    #print(space)
    return(next(item for item in space['resources'] if item['entity']["name"] == space_name)['metadata']['id'])

In [33]:
space_uid = guid_from_space_name(client, space_name='DrugClassification')
print("Space UID = " + space_uid)

NameError: name 'guid_from_space_name' is not defined

In [ ]:
client.set.default_space(space_uid)

In [ ]:
client.software_specifications.list()

In [ ]:
import sklearn
sklearn.__version__

In [ ]:
software_spec_uid= client.software_specifications.get_uid_by_name("default_py3.8")
software_spec_uid

In [ ]:
model_details = client.repository.store_model(model='drug-classification-model_new.tgz',
                                              meta_props={client.repository.ModelMetaNames.NAME:"DrugClassification",
                                                          client.repository.ModelMetaNames.TYPE:"scikit-learn_0.23",
                                                          client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_spec_uid
                                                         },
                                              training_data=x_train,
                                              training_target=y_train)

In [ ]:
model_id = client.repository.get_model_id(model_details)
model_id

In [ ]:
# Deploy
deployment = client.deployments.create(
    artifact_uid=model_id, 
    meta_props={client.deployments.ConfigurationMetaNames.NAME:"DrugClassification_deploy", 
    client.deployments.ConfigurationMetaNames.ONLINE: {}
})

In [ ]:
#'00a67b76-022e-47d1-a1d0-031cc258204f'